<a href="https://colab.research.google.com/github/Dev-hottae/PINC/blob/master/Data_Analysis/Train_KoGPT2_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **요약 모델 학습**

## **패키지 설치**

In [1]:
!ls drive/'My Drive'/'Colab Notebooks'/

 __MACOSX	  'report.ipynb의 사본의 사본'	     requirements.txt
 NarrativeKoGPT2  'report.ipynb의 사본의 사본 (1)'   Untitled0.ipynb


In [2]:
!pip install -r drive/'My Drive'/'Colab Notebooks'/requirements.txt

     |████████████████████████████████| 348kB 6.3MB/s 
     |████████████████████████████████| 55.0MB 60kB/s 
     |████████████████████████████████| 1.1MB 41.7MB/s 
     |████████████████████████████████| 1.3MB 46.2MB/s 
     |████████████████████████████████| 2.9MB 39.9MB/s 
     |████████████████████████████████| 890kB 50.5MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588515 sha256=2db21eed03aec707ceccfc9ee579d9749f1f7ff7540561eaf6617172eb74cf9c
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
  Created wheel for kss: filename=kss-1.3.1-cp36-cp36m-linux_x86_64.whl size=251572 sha256=6cd0cfff4ac6c6be47a59aebef3c40c2aaeb031f9e3bc102081630fdf005b27a
  Stored in directory: /root/.cache/pip/wheels/8b/98/d1/53f75f89925cd95779824778725ee3fa36e7aa55ed26ad54a8
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=bb8b6f0d5aae81259c11c8169e67df15b

In [3]:
!pip install git+https://github.com/SKT-AI/KoGPT2.git

  Cloning https://github.com/SKT-AI/KoGPT2.git to /tmp/pip-req-build-m1p2faq4
  Running command git clone -q https://github.com/SKT-AI/KoGPT2.git /tmp/pip-req-build-m1p2faq4
  Created wheel for kogpt2: filename=kogpt2-0.1.1-cp36-none-any.whl size=14054 sha256=bca4ad23bc960739f19ff261b43368b0b7c9f23118c0cf0f2313b44922643850
  Stored in directory: /tmp/pip-ephem-wheel-cache-zln1msxm/wheels/3b/a2/30/432bb7490a2ea23a90049e6c5725f6acd7e925f1abfb3d7ddf
Successfully built kogpt2


In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## **Import 패키지**

In [5]:
import torch
from torch.utils.data import DataLoader, Dataset
from gluonnlp.data import SentencepieceTokenizer 
import gluonnlp as nlp
import pandas as pd
from tqdm import tqdm
from nltk.tokenize import sent_tokenize

In [6]:
from kogpt2.utils import download, tokenizer, get_tokenizer
from kogpt2.pytorch_kogpt2 import GPT2Config, GPT2LMHeadModel

## **GPT 확인**

In [7]:
#GPU 사용
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

## **KoGPT-2 config**

In [8]:
ctx= 'cuda'#'cuda' #'cpu' #학습 Device CPU or GPU. colab의 경우 GPU 사용
cachedir='~/kogpt2/' # KoGPT-2 모델 다운로드 경로
epoch =200  # 학습 epoch
save_path = '/content/drive/My Drive/머신러닝/팀 프로젝트/06. AI를 이용한 금융 보고서/Data_Analysis/checkpoint/'
#use_cuda = True # Colab내 GPU 사용을 위한 값

pytorch_kogpt2 = {
    'url':
    'https://kobert.blob.core.windows.net/models/kogpt2/pytorch/pytorch_kogpt2_676e9bcfa7.params',
    'fname': 'pytorch_kogpt2_676e9bcfa7.params',
    'chksum': '676e9bcfa7'
}
kogpt2_config = {
    "initializer_range": 0.02,
    "layer_norm_epsilon": 1e-05,
    "n_ctx": 1024,
    "n_embd": 768,
    "n_head": 12,
    "n_layer": 12,
    "n_positions": 1024,
    "vocab_size": 50000,
    "output_past": None
}

## **모델, Vocab 다운로드**

In [9]:
# download model
model_info = pytorch_kogpt2
model_path = download(model_info['url'],
                       model_info['fname'],
                       model_info['chksum'],
                       cachedir=cachedir)
# download vocab
vocab_info = tokenizer
vocab_path = download(vocab_info['url'],
                       vocab_info['fname'],
                       vocab_info['chksum'],
                       cachedir=cachedir)

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [10]:
# KoGPT-2 언어 모델 학습을 위한 GPT2LMHeadModel 선언
kogpt2model = GPT2LMHeadModel.from_pretrained(pretrained_model_name_or_path=None,
                                              config=GPT2Config.from_dict(kogpt2_config),
                                              state_dict=torch.load(model_path))

device = torch.device(ctx)
kogpt2model.to(device)
# Fine Tunning을 위해 train 선언
kogpt2model.train()
# 단어 뭉치 가져오기
vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_path,
                                                     mask_token=None,
                                                     sep_token=None,
                                                     cls_token=None,
                                                     unknown_token='<unk>',
                                                     padding_token='<pad>',
                                                     bos_token='<s>',
                                                     eos_token='</s>')

In [11]:
tok_path = get_tokenizer()
vocab = vocab_b_obj
sentencepieceTokenizer = SentencepieceTokenizer(tok_path, 0, 0)

using cached model


## **데이터 불러오기**

In [12]:
data_path = '/content/drive/My Drive/머신러닝/팀 프로젝트/06. AI를 이용한 금융 보고서/Data_Analysis/DataSet/Topic_keywords.csv'
news_data = pd.read_csv(data_path, encoding='utf-8')
news_data.head()

,Date,Topic_keywords,Text
0,2006-06-27,하이닉스 매각 이번 신주 발행 만주 외국 권단 차익 증자,하이닉스반도체가 블록딜을 마치자마자 주가가 급등했다.하이닉스 주가는 26일 유가증권...
1,2006-06-27,외국인 순매도 매도 이날 전날 증시 개인 지수 투자자 매수,코스피지수가 엿새 만에 1240선을 회복했다.27일 코스피지수는 전날 미국 증시 반...
2,2006-06-27,상승 종목 이날 하락 지수 순매도 업종 프로그램 외국인 매수세,코스피지수가 하락 하루 만에 반등하며 1240선에 다가섰다.26일 코스피지수는 전거...
3,2006-06-27,창업 벤처 미국 대표 원자 한국 장비 스탠퍼드 귀국 세계,세계시장에 나갔을 때 '코리아 디스카운트'가 이 정도로 심할 줄은 몰랐습니다.그러나...
4,2006-06-27,시장 인상 금리 예상 지수 증가 상승 미국 수출 연방,"일본 닛케이225지수는 직전 저점 대비 6.7% 상승, 20일이동평균선을 회복했다...."


In [13]:
dataset_train = []
for i in tqdm(range(len(news_data))):
    dataset_train.append(news_data['Topic_keywords'][i] + ". " + news_data['Text'][i])

dataset_train[:5]

100%|██████████| 100/100 [00:00<00:00, 27861.72it/s]


['하이닉스 매각 이번 신주 발행 만주 외국 권단 차익 증자. 하이닉스반도체가 블록딜을 마치자마자 주가가 급등했다.하이닉스 주가는 26일 유가증권시장에서 전날보다 9.23% 급등한 2만9600원에 마감했다.지난 23일 옛 채권단이 구주를 매각하고 신주 발행을 통한 증자를 단행하면서 오버행 이슈가 사라지면서 빚어진 결과다.특히 이번 매각 과정에서 국내는 물론 외국 기관 수요가 많아 물량을 받아가지 못한 투자자들이 매수세를 보였다.하지만 일부 단기 차익을 노린 외국투자자들은 이날 단기 차익실현에 나서는 모습도 보였다.하이닉스 옛 채권단은 이번 매각에서 구주 4300만주를 국내외에 매각했다.하이닉스는 신주 1080만주를 주식예탁증서 형태로 외국에서 발행해 자금을 조달했다.채권단은 국내에서 1600만주, 국외에서 2700만주를 팔아 모두 1조1420억원어치를 거둬들였다.하이닉스는 이번 유상증자를 통해 3억달러를 조달했다.김장열 현대증권 연구원은 이번 지분 매각과 신주 발행으로 채권단 지분비율은 40%로 낮아지고 유동주식비율은 종전 50%에서 60%로 높아졌다 며 앞으로 추가적인 신주 발행도 이뤄질 수 있다 고 내다봤다.',
 '외국인 순매도 매도 이날 전날 증시 개인 지수 투자자 매수. 코스피지수가 엿새 만에 1240선을 회복했다.27일 코스피지수는 전날 미국 증시 반등으로 투자심리가 개선되면서 전날보다 9.49포인트 오른 1247.54로 장을 마쳤다.전날 뉴욕증시는 대형기업간 잇따른 인수ㆍ합병 소식이 전해지면서 사흘 만에 반등에 성공했다.이날 국내 증시는 외국인과 개인의 동반 매수세에 힘입어 장중 한때 1250선을 넘어서기도 했지만 미국 연방공개시장위원회 회의를 앞두고 투자자들의 관망세가 나타나면서 상승폭은 제한됐다.특히 장 마감을 앞두고 외국인이 매도우위로 돌아서며 15일째 순매도를 이어간 점이 부정적 영향을 미쳤다.외국인은 이날 409억원 순매도를 기록한 반면 개인과 기관은 각각 51억원, 195억원을 순매수했다.이와 관련해 외국인들의 매도 공세가 다소 진정될 

### **DataSet 형태 설정**

In [14]:
class GPTDataset(Dataset):
  def __init__(self, data_file, vocab, tokenizer):
    self.data =[]
    self.vocab = vocab
    self.tokenizer = tokenizer

    for data in data_file:
        tokenized_line = self.tokenizer(data)
        if len(tokenized_line) <= 1020: # 문장 총길이 1022로 제한
            index_of_words = [vocab.bos_token] + tokenized_line + [vocab.padding_token] * (1020 - len(tokenized_line)) + [vocab.eos_token]
            self.data.append(vocab(index_of_words))

  def __len__(self):
    return len(self.data)

  def __getitem__(self,index):
    item = self.data[index]
    return item

In [15]:
batch_size = 2 # 배치 사이즈 설정
news_dataset = GPTDataset(dataset_train, vocab, sentencepieceTokenizer)  # Torch DataLoader 형태 맞춰주는 Dataset 설정
news_data_loader = DataLoader(news_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=3)

### **파라미터 설정**

In [16]:
learning_rate = 1e-5
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(kogpt2model.parameters(), lr=learning_rate)

### **모델 FineTunning**

In [ ]:
epoch = 50
for epoch in range(epoch):
    count = 0
    avg_loss = (0.0, 0.0)
    for data in tqdm(news_data_loader):
        optimizer.zero_grad()
        # Data에 Torch 스택
        data = torch.stack(data)
        data = data.transpose(1,0)
        # 데이터와 모델에 GPU 설정
        data = data.to(device)
        kogpt2model = kogpt2model.to(device)
        # 결과값
        outputs = kogpt2model(data, labels=data)
        loss, logits = outputs[:2]
        loss = loss.to(device)
        loss.backward()
        avg_loss = (avg_loss[0] * 0.99 + loss, avg_loss[1] * 0.99 + 1.0)
        optimizer.step()
        count+=1

        if count % 10 == 0:
            print('epoch no.{0} train no.{1}  loss = {2:.5f} avg_loss = {3:.5f}' . format(epoch+1, count, loss, avg_loss[0] / avg_loss[1]))

    # 10에폭 단위 체크포인트 저장
    if epoch % 10 == 0:
        torch.save({
            'model_state_dict': kogpt2model.state_dict(),
            'optimizer_state_dict' : optimizer.state_dict(),
            'loss' : loss
            }, save_path+'narrativeKoGPT2_checkpoint{}.tar'.format(epoch//10))